In [38]:
import pandas as pd
from g4f.client import Client
import re
import os
import json

In [39]:
def load_data_from_folder(folder_path: str) -> pd.DataFrame:
    """
    加載資料夾中的所有 JSON 文件並合併為一個 DataFrame。
    Args:
        folder_path (str): 資料夾路徑。
    Returns:
        pd.DataFrame: 合併後的數據框。
    """
    try:
        all_data = []
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".json"):  # 檢查文件是否為 JSON 格式
                file_path = os.path.join(folder_path, file_name)
                with open(file_path, "r", encoding="utf-8") as file:
                    raw_data = json.load(file)
                    all_data.extend(raw_data)  # 將每個文件的數據添加到列表中

        # 合併所有數據為 DataFrame
        return pd.DataFrame(all_data)
    except Exception as e:
        raise Exception(f"讀取資料夾中的 JSON 數據時發生錯誤：{e}")


In [40]:
def generate_summary(data: pd.DataFrame):
    """
    生成數據描述性統計。
    Args:
        data (pd.DataFrame): 輸入數據框。
    Returns:
        pd.DataFrame: 描述性統計。
    """
    try:
        return data.describe()
    except Exception as e:
        raise Exception(f"生成描述性統計時發生錯誤：{e}")


In [41]:
def construct_prompt(user_input: str, summary_stats: pd.DataFrame) -> tuple:
    """
    構建系統與用戶提示。
    Args:
        user_input (str): 用戶查詢。
        summary_stats (pd.DataFrame): 描述性統計。
    Returns:
        tuple: (系統提示, 用戶提示)
    """
    system_prompt = (
        "你是一位專業的數據科學家，擅長分析數據並從中提取見解。"
        "以下是一些統計數據，請根據這些數據提供深入分析和建議：\\n"
        "1. 描述數據的整體特性，例如平均值、標準差、數據分佈等。\\n"
        "2. 尋找可能的異常值或數據問題，並評估其對分析結果的影響。\\n"
        "3. 如果有類別型數據，請分析其分佈特性和可能的相關性。\\n"
        "4. 根據數據，提供有助於業務決策的具體建議，例如潛在的改進方向或應關注的特定方向。\\n"
        "5. 將您的分析結論簡潔清楚地表達，並建議下一步的數據收集或分析方向。"
    )
    user_prompt = (
        f"以下是數據的描述性統計結果：\\n{summary_stats.to_string()}\\n"
        f"請根據這些數據回答以下問題：{user_input}"
    )
    return system_prompt, user_prompt
    

In [42]:

def load_data(file_path: str) -> pd.DataFrame:
    """
    JSON 文件並轉換為 DataFrame。
    """
    try:
        import json
        with open(file_path, "r", encoding="utf-8") as file:
            raw_data = json.load(file)
        return pd.DataFrame(raw_data)
    except json.JSONDecodeError as e:
        raise Exception(f"JSON 文件解析錯誤：{e}")
    except Exception as e:
        raise Exception(f"讀取數據時發生錯誤：{e}")


In [43]:
def generate_report_from_folder(folder_path: str, user_input: str) -> str:
    """
    基於資料夾中的所有 JSON 數據生成分析報告。
    Args:
        folder_path (str): 資料夾路徑。
        user_input (str): 用戶的自然語言查詢。
    Returns:
        str: 分析報告或錯誤信息。
    """
    try:
        # 1. 加載資料夾中的數據
        data = load_data_from_folder(folder_path)

        # 2. 生成描述性統計
        summary_stats = generate_summary(data)

        # 3. 構建 GPT 模型請求的提示
        system_prompt, user_prompt = construct_prompt(user_input, summary_stats)

        # 4. 初始化 GPT 客戶端並生成報告
        client = Client()
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.1,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"生成報告時發生錯誤：{e}"


In [48]:
if __name__ == "__main__":
    # 指定資料夾路徑
    folder_path = r"C:\Project_Tibame_Capstone\Project\flask_app\LLM4report\json.ingredient"
    user_input = "請分析資料夾中的數據並提供詳細見解與建議800字以上。"

    try:
        report = generate_report_from_folder(folder_path, user_input)
        print("生成的報告：")
        print(report)
    except Exception as e:
        print(f"程序運行失敗：{e}")

生成的報告：
根據提供的描述性統計數據，我們可以對數據進行深入分析，並提出相應的見解和建議。以下是對數據的詳細分析：

### 1. 整體特性分析

首先，數據的整體特性可以從平均值、標準差和數據分佈等方面進行描述：

- **年份**：數據範圍從2005年到2024年，顯示出一個相對長期的趨勢。平均年份為2014.88，表明數據主要集中在2010年代中期。
  
- **現金比率**：平均值為0.3075，標準差為0.0056，顯示出現金比率的變化相對穩定。最小值和最大值分別為0.3000和0.3150，表明現金比率在這段時間內變化不大，可能反映出企業在流動性管理上的穩定性。

- **存貨周轉率**：平均值為5.15，標準差為0.1125，顯示出企業在存貨管理上的效率。存貨周轉率的範圍從5.00到5.30，表明企業在存貨管理上保持了一定的穩定性。

- **營業利潤率**：平均值為36.4%，顯示出企業的盈利能力相對較高。標準差為0.6954，顯示出一定的波動性，可能受到市場環境或內部管理的影響。

- **年度營業利益率**：平均值為35.75%，顯示出企業的整體盈利能力。標準差為0.5625，顯示出一定的穩定性。

- **淨利潤率**、**資產回報率**和**股東權益回報率**的平均值分別為30.6%、12.45%和20.9%，這些指標顯示出企業在盈利能力和資本運用效率方面的表現。

### 2. 異常值和數據問題

在分析數據時，尋找異常值是非常重要的。根據提供的數據，沒有明顯的異常值，但需要注意以下幾點：

- **年份的分佈**：數據集中在2010年代，未來的數據（2023年和2024年）可能會影響整體趨勢的解釋。建議在未來的分析中持續更新數據，以便更好地捕捉市場變化。

- **現金比率的穩定性**：雖然現金比率變化不大，但如果未來出現大幅波動，可能會影響企業的流動性和風險管理。

### 3. 類別型數據分析

在這組數據中，年份和季度可以視為類別型數據。分析其分佈特性和可能的相關性：

- **年份**：隨著年份的增加，企業的營業利潤率和淨利潤率有上升的趨勢，這可能與市場環境的改善或企業內部管理的優化有關。

- **季度**：如果有季度數據，建議進一步分析季度間的變化，這可以幫助企業了解季節性因素對業務的影響。

### 4. 業務決策建議
